In [30]:
import os
import time
import re
from datetime import datetime
import json

import numpy as np
import pandas as pd
import plotly.express as px

In [31]:
KUDAGO_DATA_PATH = '/mnt/ess_storage/DN_1/storage/home/akorneev/new_kudago'
INST_DATA_PATH = '/mnt/ess_storage/DN_1/storage/home/akorneev/instagram_posts'

In [32]:
os.listdir(KUDAGO_DATA_PATH), os.listdir(INST_DATA_PATH)

(['403-497_clean.csv', '1-402_clean.csv', '497-1653_clean.csv'],
 ['spb_posts_2018.csv',
  'nyc_posts_2018.csv',
  'nyc_posts_2019.csv',
  'spb_posts_2019.csv',
  'nyc_04_02_2019_22-23.csv',
  'london_posts_2018.csv',
  'moscow_posts_2019.csv',
  'nyc_04_02_2019_11-13.csv',
  'moscow_posts_2018.csv',
  'london_posts_2019.csv'])

# Data preparation

## KudaGo

In [33]:
kuda_df = pd.concat([ pd.read_csv(KUDAGO_DATA_PATH + '/' + kuda_table, delimiter=';', encoding='cp1251') for kuda_table in os.listdir(KUDAGO_DATA_PATH)],
                    ignore_index=True)

In [34]:
kuda_df.shape

(165565, 12)

In [35]:
# drop incorrect rows
print(kuda_df.iloc[12623], '\n\n' , kuda_df.iloc[13653])
kuda_df = kuda_df.drop(12623)
kuda_df = kuda_df.drop(13653)

Short name           {'start': 1603645200, 'end': 1603645200}, {'s...
Name                                                              msk
Description                                                  ['none']
Body text                                                    ['none']
Dates                                                   entertainment
City                интересное,comedy club,free,юмористы,это лето,...
Place                                                             NaN
Coordinates                                                       NaN
Category                                                          NaN
Event tag                                                         NaN
['description']                                                   NaN
['body', 'text']                                                  NaN
Name: 12623, dtype: object 

 Short name           {'start': 1576602000, 'end': 1576602000}, {'s...
Name                                                        

In [36]:
kuda_df.Dates = kuda_df.Dates.apply(lambda x: x if x != '[]' else np.nan)
kuda_df.Coordinates = kuda_df.Coordinates.apply(lambda x: x if x != 'None' else np.nan)

In [37]:
# drop rows with empty space and date columns
kuda_df = kuda_df.dropna(subset = ['Dates', 'Coordinates'])
kuda_df.shape

(136478, 12)

Events with multiple dates -> different rows

In [38]:
def obj_to_list(x):
    if x == "[]":
        return np.nan
    elif isinstance(x, float) and np.isnan(x):
        return np.nan
    
    x = x.replace('[', '').replace(']', '').replace('},', '};')
    return x.split(';')

In [39]:
kuda_df.Dates = kuda_df.Dates.apply(obj_to_list)

In [40]:
kuda_df = kuda_df.explode("Dates").reset_index()

In [41]:
kuda_df.tail(5)

,index,Short name,Name,Description,Body text,Dates,City,Place,Coordinates,Category,Event tag,['description'],"['body', 'text']"
368779,165563,Евгений Петросян и Елена Степаненко,концерт Евгения Петросяна и Елены Степаненко,NaN,NaN,"{'start': 1425819600, 'end': 1425826800}",msk,"ул. Лесная, д. 18","{'lat': 55.77899799999997, 'lon': 37.590300000...","concert,entertainment","шоу,петросян,16+,концерты,новогодние концерты,...","['фееричный', 'юмористический', 'шоу', 'популя...","['один', 'самый', 'востребовать', 'сегодня', '..."
368780,165563,Евгений Петросян и Елена Степаненко,концерт Евгения Петросяна и Елены Степаненко,NaN,NaN,"{'start': 1428249600, 'end': 1428256800}",msk,"ул. Лесная, д. 18","{'lat': 55.77899799999997, 'lon': 37.590300000...","concert,entertainment","шоу,петросян,16+,концерты,новогодние концерты,...","['фееричный', 'юмористический', 'шоу', 'популя...","['один', 'самый', 'востребовать', 'сегодня', '..."
368781,165563,Евгений Петросян и Елена Степаненко,концерт Евгения Петросяна и Елены Степаненко,NaN,NaN,"{'start': 1451926800, 'end': 1451934000}",msk,"ул. Лесная, д. 18","{'lat': 55.77899799999997, 'lon': 37.590300000...","concert,entertainment","шоу,петросян,16+,концерты,новогодние концерты,...","['фееричный', 'юмористический', 'шоу', 'популя...","['один', 'самый', 'востребовать', 'сегодня', '..."
368782,165563,Евгений Петросян и Елена Степаненко,концерт Евгения Петросяна и Елены Степаненко,NaN,NaN,"{'start': 1483804800, 'end': 1483812000}",msk,"ул. Лесная, д. 18","{'lat': 55.77899799999997, 'lon': 37.590300000...","concert,entertainment","шоу,петросян,16+,концерты,новогодние концерты,...","['фееричный', 'юмористический', 'шоу', 'популя...","['один', 'самый', 'востребовать', 'сегодня', '..."
368783,165564,NaN,выставка «Мемориальная мастерская Т.Г. Шевченко»,NaN,NaN,"{'start': 1415437200, 'end': 1415901600}",spb,"ул. Шамшева, д. 8","{'lat': 59.96041644411368, 'lon': 30.300563867...",exhibition,"выставки,история и личности","['единственный', 'место', 'санкт', 'петербург'...","['мастерская', 'представлять', 'себя', 'уютный..."


## Instagram

In [28]:
inst_spb_2018 = pd.read_csv(INST_DATA_PATH + '/' + 'spb_posts_2018.csv')

In [15]:
inst_msk_2018 = pd.read_csv(INST_DATA_PATH + '/' + 'moscow_posts_2018.csv')

# Datasets connection

## Extracting 'start' and 'end' dates for KudaGo

In [42]:
def get_value(x, val, val_type):
    if x == "[]":
        return ""
    elif isinstance(x, float) and np.isnan(x):
        return ""
    
    try:
        json_parsed = json.loads(x.replace("'", '"').replace('None', '"None"'))
    except:
        print(type(x), x)
        
    try:
        answer = val_type( json_parsed[val] )
    except:
        answer = np.nan
        
    return answer

In [43]:
kuda_df['Start_date'] = kuda_df.Dates.apply(get_value, args = ['start', int])

In [44]:
kuda_df['End_date'] = kuda_df.Dates.apply(get_value, args = ['end', int])

In [45]:
kuda_df['lat'] = kuda_df.Coordinates.apply(get_value, args = ['lat', float])

In [46]:
kuda_df['lon'] = kuda_df.Coordinates.apply(get_value, args = ['lon', float])

In [47]:
print(kuda_df.shape)
kuda_df = kuda_df.dropna(subset = ['lat', 'lon', 'Start_date', 'End_date'])
print(kuda_df.shape)

(368784, 17)
(368716, 17)


## Instagram and KudaGo connection

In [48]:
# Inst data exists for 2018 - 2020 (till april) years for spb and msk, cutting the rest
begin = int(time.mktime(datetime(2018, 1, 1, 0, 0).timetuple()))
finish = int(time.mktime(datetime(2020, 5, 1, 0, 0).timetuple()))
begin, finish

(1514764800, 1588291200)

In [49]:
#ts = int('1514764800')
#print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

In [50]:
kuda_msk = kuda_df[(kuda_df.Start_date > 1514764800) & (kuda_df.End_date < 1588291200) & (kuda_df.City == 'msk')]
kuda_spb = kuda_df[(kuda_df.Start_date > 1514764800) & (kuda_df.End_date < 1588291200) & (kuda_df.City == 'spb')]
kuda_msk.shape, kuda_spb.shape

((49540, 17), (37890, 17))

In [51]:
"""
kuda_row - row of KudaGo data with "lon", "lat", "Start_date", "End_date" columns
Inst_df - Instagram df with "timestamp", "lat", "lon"
time_shift - UNIX timestamp, temporal shift (in seconds)
space_shift - Degrees, sparial shift

"""
def connect_Kuda2Inst(kuda_row, inst_df, time_shift, space_shift, spatial_type='rectangle'):
    # temporal filtering
    posts = inst_df[(inst_df.timestamp > ( kuda_row.Start_date - time_shift )) & (inst_df.timestamp < ( kuda_row.End_date + time_shift ))]
    
    # spatial filtering
    if spatial_type == 'rectangle':
        posts = posts[(posts.lat > ( kuda_row.lat - space_shift )) & (posts.lat < ( kuda_row.lat + space_shift ))]
        posts = posts[(posts.lon > ( kuda_row.lon - space_shift )) & (posts.lon < ( kuda_row.lon + space_shift ))]   
    elif spatial_type == 'circle':
        raise ValueError('Circle type is not supported for now')
    else:
        raise ValueError('Unknown spatial type value')
                                                                 
    return list(posts.shortcode), posts

## testing

In [73]:
kuda_msk.iloc[2]

index                                                               5
Short name                                               Пиковая дама
Name                спектакль «Пиковая дама» в постановке «Коляда-...
Description         ['оригинальный', 'вариация', 'пиковый', 'дама'...
Body text           ['пиковый', 'дама', 'коляда', 'дерзкий', 'авто...
Dates                        {'start': 1516204800, 'end': 1516204800}
City                                                              msk
Place                                            Страстной б-р, д. 8а
Coordinates              {'lat': 55.76625299999996, 'lon': 37.610826}
Category                                                      theater
Event tag           экспериментальный театр,современная драматурги...
['description']                                                   NaN
['body', 'text']                                                  NaN
Start_date                                                 1516204800
End_date            

In [64]:
answer = connect_Kuda2Inst(kuda_msk.iloc[2], inst_msk_2018, 60 * 60, 0.005, 'rectangle')
answer[0]

['BeDtvYmlvll',
 'BeDrSSODByU',
 'BeDn6e-g7b9',
 'BeDnrKaD9pb',
 'BeDnQeTDNRP',
 'BeDm5j_h7Ms',
 'BeDmpujFPYG',
 'BeDmhMQFgLh',
 'BeDhcRwFsXH']

In [66]:
answer[1]

,id,shortcode,imageurl,isvideo,caption,commentscount,timestamp,likescount,isad,authorid,locationid,lat,lon
3103671,1694399053147076965,BeDtvYmlvll,https://scontent-iad3-1.cdninstagram.com/v/t51...,f,#юнонаиавось,0,1516208129,29,f,1524847225,500210865,55.767713,37.606409
3103672,1694388257335024788,BeDrSSODByU,https://scontent-iad3-1.cdninstagram.com/v/t51...,f,Михаил Пуговкин:\n🔴Когда Станиславского спроси...,50,1516206842,2964,f,3851522673,500210865,55.767713,37.606409
3103673,1694373427626096381,BeDn6e-g7b9,https://scontent-iad3-1.cdninstagram.com/v/t51...,f,Прямо сейчас в ожидании легендарной рок-оперы ...,0,1516205074,62,f,2242753534,500210865,55.767713,37.606409
3103674,1694372374747535963,BeDnrKaD9pb,https://scontent-iad3-1.cdninstagram.com/v/t51...,f,Сегодня в программе - культпоход ❤️ #ленком #l...,0,1516204948,10,f,393445001,500210865,55.767713,37.606409
3103675,1694370540678861903,BeDnQeTDNRP,https://scontent-iad3-1.cdninstagram.com/v/t51...,f,Юнона и авось. Я так давно хотел это увидеть. ...,4,1516204730,72,f,3452123501,500210865,55.767713,37.606409
3103676,1694368966245856044,BeDm5j_h7Ms,https://scontent-iad3-1.cdninstagram.com/v/t51...,f,С надеждой на окультуривание направились в Лен...,6,1516204542,1175,f,18087245,500210865,55.767713,37.606409
3103677,1694367878068106758,BeDmpujFPYG,https://scontent-iad3-1.cdninstagram.com/v/t51...,f,Сегодня очень культурная программа *Юнона и Ав...,12,1516204412,44,f,2102947433,500210865,55.767713,37.606409
3103678,1694367291486372577,BeDmhMQFgLh,https://scontent-iad3-1.cdninstagram.com/v/t51...,f,#юнонаиавось,2,1516204342,68,f,507587846,500210865,55.767713,37.606409
3107015,1694344963562063303,BeDhcRwFsXH,https://instagram.frix7-1.fna.fbcdn.net/v/t51....,f,Keep going! 🏃🏻‍♀️🏃🏻‍♀️\nПочему бы не взять за ...,9,1516201681,692,f,460922071,269321222,55.767030,37.607820


## MSK dataset

In [76]:
msk_connections = dict()
counter = 0
for index, row in kuda_msk.iterrows():
    if counter % 1000 == 0 and counter != 0:
        print(counter, '/', kuda_msk.shape[0])
        break
    counter += 1
    answer = connect_Kuda2Inst(row, inst_msk_2018, 60 * 60, 0.01, 'rectangle')[0]
    if len(answer) > 0:
        msk_connections[row[0]] = answer

1000 / 49540


In [78]:
msk_connections[2]

['Bn4DVv_nytn',
 'Bn4Fol_jbjR',
 'Bn4FWKhjtmK',
 'Bn4ALtqnb9K',
 'Bn4Dy5Sn-tI',
 'Bn4DhpnFCUm',
 'Bn4GSiHlZb6',
 'Bn4GIvDHlwC',
 'Bn4G-kAAOVJ',
 'Bn4CD23gSgB',
 'Bn4A8JJhIp0',
 'Bn4AlslhUrf',
 'Bn4C4LkjL3a',
 'Bn4BH2bA1iw',
 'Bn4AvKEh_WY',
 'Bn3-Dw3DOdF']

### checking

In [845]:
kuda_msk.iloc[28]

index                                                              28
Short name                             Евгений Онегин. Своими словами
Name                спектакль «Евгений Онегин. Своими словами» в Т...
Description         ['спектакль', 'ребёнок', 'евгений', 'онегин', ...
Body text           ['создатель', 'постановка', 'вовсе', 'намерить...
Dates                        {'start': 1560963600, 'end': 1560969000}
City                                                              msk
Place                                             ул. Сретенка, д. 19
Coordinates         {'lat': 55.77019711398269, 'lon': 37.631941461...
Category                                                 theater,kids
Event tag           евгений онегин,спектакли,драматические постано...
['description']                                                   NaN
['body', 'text']                                                  NaN
Start_date                                                 1560963600
End_date            

In [822]:
inst_df[inst_df.shortcode == 'By5fNQRAVMy']

,id,shortcode,imageurl,isvideo,caption,commentscount,timestamp,likescount,isad,authorid,locationid,lat,lon
2330904,2069822754013664050,By5fNQRAVMy,https://scontent-sea1-1.cdninstagram.com/v/t51...,f,Совсем недавно я давала интервью каналу @78_ch...,0,1560962121,55,f,8974270374,1616111155139848,55.72189,37.63873


In [868]:
inst_df[inst_df.shortcode == 'By5fNQRAVMy'].caption

2330904    Совсем недавно я давала интервью каналу @78_ch...
Name: caption, dtype: object

## SPB dataset

In [882]:
spb_connections = dict()
counter = 0
for index, row in kuda_spb.iterrows():
    if counter % 100 == 0 and counter != 0:
        print(counter, '/', kuda_spb.shape[0])
        break
    counter += 1
    answer = connect_Kuda2Inst(row, inst_spb_2018, 60 * 60, 0.001, 'rectangle')[0]
    if len(answer) > 0:
        spb_connections[row[0]] = answer

100 / 37890


In [883]:
spb_connections

{21: ['BvhGQ-9jhSR', 'BvhH49gFcuQ', 'BvhY481H4b5'],
 39: ['B578c1FD0q0',
  'B579yycgTnO',
  'B57-gi5JeoJ',
  'B58Kc8uoRv5',
  'B58LRofAXuD',
  'B58MCAegbnd'],
 45: ['B2FCE9EhN8J',
  'B2E4Zg8HeHq',
  'B2E5CLyIeuR',
  'B2E-zTpjSJQ',
  'B2E_v-UC_iU',
  'B2FA1KcA_QA',
  'B2FBQhpiTHD',
  'B2E6R0JjxtO',
  'B2E2OOGibMH',
  'B2E-CmbAcFa',
  'B2E3Bl7h27I'],
 74: ['B2q4xNbjMs_'],
 83: ['Bxz5v5WCcIo',
  'Bxz6hZHossE',
  'Bxz7ajoCUjh',
  'Bxz9ovYoE3r',
  'Bxz-EGLorG3',
  'Bxz-fHgIIqm',
  'Bx0AFZHCVZj',
  'Bx0COKcIHTm',
  'Bx0C3BbB4or',
  'Bx0C-NRjAla',
  'Bx0DLSEh0Fe',
  'Bx0DZE9odGh',
  'Bx0EdIYiLeU',
  'Bx0EjZxoMcC',
  'Bx0EuokiQcx',
  'Bx0FcyBJ8bn'],
 86: ['B6Ge5E9FALW'],
 92: ['BuJjRO-HRPP',
  'BuJjaw-nsTn',
  'BuJjmlsHxiN',
  'BuJlLmGFNTt',
  'BuJln_0lAXh',
  'BuJmJMaHdAH',
  'BuJldUlAep-']}

In [885]:
kuda_spb.iloc[21]

index                                                              27
Short name                                                  Жар-Птица
Name                спектакль «Жар-Птица» в Детском драматическом ...
Description         ['премьера', 'волшебный', 'спектакль', 'мотив'...
Body text           ['начаться', 'спектакль', 'новогодний', 'интер...
Dates                        {'start': 1547379000, 'end': 1547379000}
City                                                              spb
Place                                            пер. Советский, д. 5
Coordinates              {'lat': 59.914047, 'lon': 30.30917599999999}
Category                                                 theater,kids
Event tag           спектакли,драматические постановки,снегурочка,...
['description']                                                   NaN
['body', 'text']                                                  NaN
Start_date                                                 1547379000
End_date            

In [886]:
inst_df[inst_df.shortcode == 'BvhGQ-9jhSR']

,id,shortcode,imageurl,isvideo,caption,commentscount,timestamp,likescount,isad,authorid,locationid,lat,lon
1103879,2008914464179360913,BvhGQ-9jhSR,https://scontent-frx5-1.cdninstagram.com/v/t51...,f,"Честно, не платила ему за это..😂🙈🥰 .\n\nЯ похв...",11,1553701287,104,f,3064752718,498774945,59.919957,30.334787


In [891]:
inst_df[inst_df.shortcode == 'BvhY481H4b5']

,id,shortcode,imageurl,isvideo,caption,commentscount,timestamp,likescount,isad,authorid,locationid,lat,lon
1106603,2008996375506683641,BvhY481H4b5,https://scontent-frx5-1.cdninstagram.com/v/t51...,f,NaN,2,1553711052,72,f,10206269736,498774945,59.919957,30.334787
